## Section 0. Environmental Setup

### Step A. Check Tensorflow Version

針對環境，本次作業會採用 TensorFlow 進行操作，根據 Google Tensorflow 

In [23]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
print("Tensorflow Version: ", tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("Numbers of GPU Available: ", len(tf.config.list_physical_devices('GPU')))

Tensorflow Version:  2.18.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Numbers of GPU Available:  1


### Step B. Check GPU Availability and Set VRAM Growth

TensorFlow takes up all available VRAM upon startup by default, which can potentially trigger an OOM error or interfere with other system graphics applications. Consequently, in this implementation, **`memory_growth`** is set to **`True`**. This configuration allows VRAM to be allocated dynamically by TensorFlow based on actual demand, instead of immediately hogging the full **8 GiB** of space.

In [24]:
print("Name:", gpu.name, " ; Type:", gpu.device_type)
tf.config.experimental.set_memory_growth(gpu, True)

Name: /physical_device:GPU:0  ; Type: GPU


## Section 1. Data Engineering and Statistical Verification

> _The following procedure is structured in the form of Casella & Berger (2001) Statistical Inference.

Define $\mathcal{D} = \{ (x_i, y_i, s_i) \}_{i=1}^N$ as the complete dataset, specifically the DataFrame `df`, where:

- $x_i \in \mathcal{X}$ is the $i$-th MRI slice image (Tensor / Array).
- $y_i \in \mathcal{Y}$ is the class label `class_name`, where $\mathcal{Y} = \{ \text{ND, VMD, MD, MOD} \}$.
- $s_i \in \mathcal{S}$ is the subject identifier `sid`, representing patient $P$.
- $f(x)$ is the joint probability density function.

### Step A. Metadata of the DataFrame

Given the directory structure (excluding images), where `work.ipynb` is the Jupyter Notebook for this operation:

```text
.
├── README.md
├── work.ipynb
└── oasis_data
    ├── dementia_mild
    ├── dementia_moderate
    ├── dementia_very_mild
    └── non-demented

6 directories, 2 files
```

Since the filenames in the OASIS dataset are typically structured as `OAS1_0028_MR1_mpr-1_100.jpg` with `OAS1_0028` being the Subject ID,it is first necessary to maintain the original structure while extracting the ID using a regexp.

In [3]:
import re

def extract_sid(filename):
    match = re.search(r"(OAS\d+_\d+)", filename)
    if match:
        return match.group(1)
    else:
        return "Unknown"

Next, scanning of all files can be performed, and the DataFrame `df` subsequently created.

In [25]:
import os
import glob
import pandas as pd

CLASSES = ["non-demented", "dementia_very_mild", "dementia_mild", "dementia_moderate"]
DATA_ROOT = "oasis_data"

data_list = []

for label in CLASSES:
    files = glob.glob(os.path.join(DATA_ROOT, label, "*.jpg"))
    
    for file_path in files:
        file_name = os.path.basename(file_path)
        sid = extract_sid(file_name)
        data_list.append({
            "file_path": file_path,
            "file_name": file_name,
            "sid": sid,
            "class_name": label
        })

df = pd.DataFrame(data_list)
print(df)

                                               file_path  \
0      oasis_data/non-demented/OAS1_0001_MR1_mpr-1_10...   
1      oasis_data/non-demented/OAS1_0001_MR1_mpr-1_10...   
2      oasis_data/non-demented/OAS1_0001_MR1_mpr-1_10...   
3      oasis_data/non-demented/OAS1_0001_MR1_mpr-1_10...   
4      oasis_data/non-demented/OAS1_0001_MR1_mpr-1_10...   
...                                                  ...   
74971  oasis_data/dementia_moderate/OAS1_0351_MR1_mpr...   
74972  oasis_data/dementia_moderate/OAS1_0351_MR1_mpr...   
74973  oasis_data/dementia_moderate/OAS1_0351_MR1_mpr...   
74974  oasis_data/dementia_moderate/OAS1_0351_MR1_mpr...   
74975  oasis_data/dementia_moderate/OAS1_0351_MR1_mpr...   

                         file_name        sid         class_name  
0      OAS1_0001_MR1_mpr-1_100.jpg  OAS1_0001       non-demented  
1      OAS1_0001_MR1_mpr-1_101.jpg  OAS1_0001       non-demented  
2      OAS1_0001_MR1_mpr-1_102.jpg  OAS1_0001       non-demented  
3      OAS1

Now let's check the data.

1. Check the total number of images and subjects.
2. Check the distribution of images per class.
3. Check the distribution of subjects per class.

In [ ]:
# 1. Check the total number of images and subjects
print(f"Total Images: {len(df)}")
print(f"Total Subjects: {df['sid'].nunique()}")
print()

# 2. Check the distribution of images per class
print("Images per Class: \n", df['class_name'].value_counts())
print()

# 3. Check the distribution of images per subject
print("Images per Subject (Top 5): \n", df['sid'].value_counts().head())

Total Images: 74976
Total Subjects: 300

Images per Class: 
 class_name
non-demented          67222
dementia_mild          5002
dementia_very_mild     2264
dementia_moderate       488
Name: count, dtype: int64

Images per Subject (Top 5): 
 sid
OAS1_0379    488
OAS1_0353    488
OAS1_0061    488
OAS1_0368    488
OAS1_0285    488
Name: count, dtype: int64


### Step B. Establishing the i.i.d. Condition

In **Classical Statistics** (1700s–1960s) for parameter estimation, common methods include **Maximum Likelihood Estimation (MLE)**, **Point Estimation (PE)**, **Bayesian Estimation (BE)**, **Uniformly Most Powerful Test (UMP)**, **Uniformly Minimum Variance Unbiased Estimator (UMVUE)**, and so on. In principle, all statistical inferences are **entirely built upon the premise that "the data are i.i.d."**

> **Definition 5.1.1** Identical and Independent Distribution
>
> The random variables $\{X_i\}_{i=1}^{n}$ are called a **random sample** of size $n$ from the **population** $f(x)$ if $\{X_i\}_{i=1}^{n}$ are mutually independent random variables and the marginal **pdf** or **pmf** of each $X_i$ is the same function $f(x)$.
>
> Alternatively, $\{X_i\}_{i=1}^{n}$ are called **independent and identically distributed** random variables with **pdf** or **pmf** $f(x)$. This is commonly abbreviated to **i.i.d.** random variables.
>

This **i.i.d.** assumption makes the optimization formulas very tractable mathematically. Furthermore, when the sample size is sufficiently large, the Central Limit Theorem dictates that the optimization approach most likely to yield the maximum likelihood is statistically correct. In other words, when sampling from some (joint) probability distribution $P$, as long as the **i.i.d.** assumption holds, the **Law of Large Numbers** and **Uniform Convergence** will be satisfied.

Today, due to the increasing complexity of input data, the true distribution of the data is often difficult to ascertain, and a large portion of data will violate the **i.i.d.** assumption. Even if the distribution family is known, calculation is challenging under extremely high-dimensional parameter spaces, causing all classical theories to fail. Consequently, a method is needed that **does not require assuming a data distribution and can perform optimization solely based on the data itself**. The concept proposed by Vapnik, V. N., & Chervonenkis, A. Y. (1971), **Empirical Risk Minimization (ERM)**, is fundamentally based on the idea that:

> *Since the true joint probability distribution $P(X, Y)$ is unknown, under the condition of having a large amount of sample data, **minimizing the average error on the samples** can be used as an alternative optimization objective.*
>

By definition, let the input space be $\mathcal{X}$ and the output space be $\mathcal{Y}$, and assume they follow some unknown joint probability distribution $P(X, Y)$. Given a loss function $L(f(x), y)$, the theoretical goal of machine learning is to find a function $f^*$ that minimizes the **Expected Risk (True Risk)**:

$$
R(f) = E_{(x,y) \sim P}[L(f(x), y)] = \int L(f(x), y) \cdot dP(x, y)
$$

However, $R(f)$ cannot be directly computed because $P(X, Y)$ is unknown. Therefore, based on the **ERM principle**, the practical approach is to optimize the following objective function (including a regularization term):

$$
\hat{f} = \arg\min_f \left[ \underbrace{\frac{1}{n}\sum L(f(x_i),y_i)}_{\text{empirical risk}} + \underbrace{R(f)}_{\text{regularizer}}\right]
$$

Here, $R(f)$ is the **regularizer**, which can also be realized through **Implicit Regularization** in modern deep learning. Considering current CPU computational resources, the majority of mainstream supervised machine learning algorithms, including **Linear Regression**, **Logistic Regression**, **SVM**, and **Neural Networks** combined with the Adam optimizer, almost universally adopt **Empirical Risk Minimization** (or its Structural Risk Minimization form) as the optimization objective during training. With current computational resources, ERM serves as a **unified training principle** applicable to models ranging from hundreds of parameters to trillions of parameters (e.g., 1.8T parameters).

To ensure that $\hat{R}_n(f)$ is a **valid estimator** of $R(f)$, one must rely on the **Weak Law of Large Numbers (WLLN)** for justification. According to:

> **Theorem 5.5.2** Weak Law of Large Number
>
> Let $\{X_i\}_{i=1}^{n}$ be **i.i.d. random variables** with $E[X_i] = \mu$ and $\mathrm{Var}[X_i] = \sigma^2 < \infty$. Define $\bar{X}_n = (1/n)\sum_{i=1}^n X_i$. Then
>
> $$\forall \: \epsilon > 0, \quad \lim_{n \to \infty} P(|\bar{X} - \mu| < \epsilon) = 1$$
>
>
> that is, $\bar{X}_n$ **converges in probability** to $\mu$.

Based on this definition, the loss function $L(f(x), y)$ in machine learning can be viewed as a random variable $Z$. If the sample data $\mathcal{D} = \{(x_i, y_i)\}_{i=1}^n$ satisfies the **i.i.d.** condition, then the array of loss values $L(f(x_1), y_1), \dots, L(f(x_n), y_n)$ are also **independent and identically distributed**. In this case, according to the **WLLN**, the empirical risk will converge in probability to the expected risk:

$$
\frac{1}{n}\sum_{i=1}^n L(f(x_i), y_i) \xrightarrow{P} E_{(x,y) \sim P}[L(f(x), y)]
$$

In other words, provided the **i.i.d.** assumption holds, **ERM** is equivalent to minimizing the model's generalization error on the true distribution. Conversely, if the **i.i.d.** assumption is violated, the **WLLN** fails, the empirical risk is **no longer a consistent estimator** of the expected risk, and **all statistical guarantees** of **ERM** become invalid for the model's training results. Hurlbert, S. H. (1984) mentions that this type of issue belongs to **Simple Pseudoreplication**, where there is only one experimental unit (replicate) per treatment, but multiple samples are drawn from that unit and treated as independent repeats for statistical testing.

Therefore, the correct sampling unit must be the **single subject** $s \in \mathcal{S}$ as the experimental unit. We must assume that different subjects satisfy independence, $\mathrm{Cov}(s_i, s_j) = 0$ for $i \neq j$, and design the data splitting process based on this. Based on this definition, if the dataset $\mathcal{D} = \{X_i\}_{i=1}^{n}$ satisfies **i.i.d.**, its joint probability density function must satisfy:

$$
f(x_1, \dots, x_n) = \prod_{i=1}^n f(x_i)
$$

Now, returning to the OASIS dataset $\mathcal{D}$, we need to examine whether the **observational units (Slices)** satisfy independence. Define $x_{i, s}$ and $x_{j, s}$ as two different MRI slice images from the same subject $s \in \mathcal{S}$, where $i \neq j$. To test for independence, we need to utilize the properties of the covariance as follows:

> **Theorem 4.5.5**
>
> If $X$ and $Y$ be independent random variables, then $\mathrm{Cov}(X,Y) = 0$ and $\rho_{XY} = 0$.
>
> $$\mathrm{Cov}(X,Y) = E[XY] - E[X]E[Y] = E[X]E[Y] - E[X]E[Y] = 0$$
>

Stated differently, if $\mathrm{Cov}(X, Y) \neq 0$, then $X$ and $Y$ are **necessarily dependent**. The question now is:

> *Can the observational unit $x \in \mathcal{X}$ be used as an **i.i.d.** sampling unit?*
>

Since the anatomical structure of the human brain exhibits a high degree of continuity and subject-specific characteristics, adjacent slices from the same subject will have significant **Intra-class Correlation** in the feature space, leading to $\mathrm{Cov}(x_{i, s}, x_{j, s}) \gg 0$. Mathematically, this means their joint expected value is not equal to the product of their marginal expected values, specifically:

$$
\mathrm{Cov}(x_{i, s}, x_{j, s}) \gg 0 \implies E[x_{i, s} \cdot x_{j, s}] \neq E[x_{i, s}] \cdot E[x_{j, s}]
$$

Therefore, if the **observational unit** $x \in \mathcal{X}$ (i.e., a single image slice) is directly treated as the sampling unit, the dataset $\mathcal{D}$ **violates the independence condition of the i.i.d. assumption, rendering the WLLN inapplicable. The empirical risk** $\hat{R}_n(f)$ **is no longer a consistent estimator, and all statistical guarantees of ERM are entirely void.**

The statistical sampling methods can be categorized into:

1. **Observational Unit** $x \in \mathcal{X}$: A single 2D MRI image, which is the minimal input unit for the model.
2. **Experimental Unit** $s \in \mathcal{S}$: The patient (Subject).